In [1]:
!pip install transformers peft
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes faiss-cpu  pdfplumber langchain fastapi
!pip install ngrok langchain-community
!pip install nest_asyncio pyngrok uvicorn fastapi pdfplumber langchain transformers peft faiss-cpu
!pip install pdfplumber
!pip install python-multipart
!pip install bitsandbytes accelerate transformers peft
!pip install --upgrade typing_extensions
!ngrok authtoken 2u4194PgxRpyjctUkSl3ZtKbEqz_3RrZ7tc5wJbmPaXzWD9y1


  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.29.2-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.3-cp310-cp310-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached charset_normalizer-3.4.1-cp310-cp310-macosx_10_9_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB

In [ ]:
!huggingface-cli login 


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ^C



In [2]:
from huggingface_hub import login

login("REMOVED_TOKEN")

/Users/lijifeng/miniforge3/envs/llama_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import os
import torch
import faiss
import random
import pdfplumber
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from peft import PeftModel
# -------------------------------------------------------------------------
# 1. Set the device: use MPS if available (Apple Silicon), otherwise CPU
# -------------------------------------------------------------------------
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

# -------------------------------------------------------------------------
# 2. Function to process a bank statement PDF
# -------------------------------------------------------------------------
def process_bank_statement(file_path: str) -> str:
    """
    Extract text from a PDF bank statement.
    """
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
    return text.strip()

# -------------------------------------------------------------------------
# 3. Build a RAG database using FAISS
# -------------------------------------------------------------------------
def build_rag_database(documents: list[str], embedding_model: SentenceTransformer):
    """
    Given a list of documents, compute embeddings and build a FAISS index.
    Returns the FAISS index object.
    """
    # Encode documents and move them to CPU for FAISS
    doc_embeddings = embedding_model.encode(documents, convert_to_tensor=True).cpu().numpy()
    dimension = doc_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(doc_embeddings)
    print(f"✅ FAISS index built with {index.ntotal} documents (dimension={dimension}).")
    return index

# -------------------------------------------------------------------------
# 4. Retrieve top-k documents from FAISS
# -------------------------------------------------------------------------
def retrieve_top_k_docs(
    query: str,
    index,
    documents: list[str],
    embedding_model: SentenceTransformer,
    k=3,
    threshold=0.5,
    debug=True
) -> list[str]:
    """
    Search the FAISS index for the top-k relevant documents given the query.
    """
    query_embedding = embedding_model.encode([query], convert_to_tensor=True).cpu().numpy()
    distances, indices = index.search(query_embedding, k)
    retrieved_docs = []
    for i, dist in zip(indices[0], distances[0]):
        # Check both index validity and distance threshold
        if i < len(documents) and dist >= threshold:
            retrieved_docs.append(documents[i])
    if debug:
        print("\n🔍 Retrieved Documents:")
        if retrieved_docs:
            for idx, doc in enumerate(retrieved_docs, 1):
                print(f"{idx}. {doc[:100]}...")
        else:
            print("No relevant documents found.")
    return retrieved_docs

# -------------------------------------------------------------------------
# 5. Request type detection (using zero-shot classification)
# -------------------------------------------------------------------------
from transformers import pipeline as hf_pipeline

classifier = hf_pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def detect_request_type(query: str) -> str:
    """
    Determines whether the user's query is a bank transaction inquiry or a general inquiry.
    """
    candidate_labels = ["bank transaction inquiry", "general inquiry"]
    result = classifier(query, candidate_labels=candidate_labels)
    predicted_label = result["labels"][0]
    print(f"🔎 Detected request type: {predicted_label}")
    return predicted_label

# -------------------------------------------------------------------------
# 6. Prompt template and function to handle client requests (RAG + Llama)
# -------------------------------------------------------------------------
PROMPT_TEMPLATE = (
    "Use the following context from bank transaction documents to answer the question. "
    "Keep the answer concise (no more than three sentences) and end with 'thanks for asking!'.\n\n"
    "Context:\n{context}\n\n"
    "Question: {question}\n\n"
    "Answer:"
)

def handle_client_request(
    query: str,
    index,
    documents: list[str],
    embedding_model: SentenceTransformer,
    llama_pipeline
) -> str:
    """
    Checks if the query is about bank transactions. If yes, retrieve context from FAISS
    and build a prompt. Otherwise, just use the query as is. Then generate a response
    using the Llama pipeline.
    """
    req_type = detect_request_type(query)
    
    if req_type == "bank transaction inquiry":
        context_docs = retrieve_top_k_docs(query, index, documents, embedding_model, k=3, threshold=0.5)
        context_text = "\n".join(context_docs) if context_docs else "No relevant bank transaction data found."
        formatted_prompt = PROMPT_TEMPLATE.format(context=context_text, question=query)
    else:
        formatted_prompt = query
    
    print("\n🤖 Formatted prompt for generation:\n", formatted_prompt[:300], "...\n")
    
    # Tokenize input and move tensors to the target device
    inputs = llama_pipeline.tokenizer(formatted_prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate text (full precision by default in PyTorch)
    outputs = llama_pipeline.model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.4,
        top_p=0.7,
        repetition_penalty=1.5,
        num_return_sequences=3
    )
    responses = [llama_pipeline.tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
    unique_responses = list(set(responses))
    final_response = unique_responses[0] if unique_responses else responses[0]
    return final_response

# -------------------------------------------------------------------------
# 7. Initialize Llama model and build RAG database
# -------------------------------------------------------------------------
# Note: On MPS, bitsandbytes is not supported, so we load the full model in standard precision.
model_name = "meta-llama/Llama-3.1-8B"
peft_model_name = "FinGPT/fingpt-mt_llama3-8b_lora"
# Load the model and move to device
llama_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
llama_model.to(device)

llama_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token

llama_model = PeftModel.from_pretrained(llama_model, peft_model_name)
# Create a pipeline for text generation
llama_pipeline = pipeline("text-generation", model=llama_model, tokenizer=llama_tokenizer)

# Load a sample dataset (cc_news) and build the FAISS database
dataset = load_dataset("cc_news", split="train[:1000]")
documents = [row["text"] for row in dataset if isinstance(row["text"], str) and row["text"].strip() != ""]
print("✅ Number of valid documents in RAG database:", len(documents))

# Use a multilingual embedding model (you can switch to a domain-specific model if desired)
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
faiss_index = build_rag_database(documents, embedding_model)

# -------------------------------------------------------------------------
# 8. Interactive chat simulation
# -------------------------------------------------------------------------
def chat_with_llama():
    """
    Interactive loop: checks request type, retrieves context if needed, and
    generates answers using Llama.
    """
    print("\n💡 Welcome to the Bank Transaction RAG Chatbot on MPS! Type 'exit' to quit.\n")
    history = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("\n👋 Goodbye!")
            break
        response = handle_client_request(user_input, faiss_index, documents, embedding_model, llama_pipeline)
        if response in history:
            response = random.choice(history) if history else response
        history.append(response)
        if len(history) > 5:
            history.pop(0)
        print("Chatbot:", response)

# Start the interactive chat
chat_with_llama()

Using device: mps


Device set to use mps:0
Loading checkpoint shards: 100%|██████████| 4/4 [00:31<00:00,  7.92s/it]
/Users/lijifeng/miniforge3/envs/llama_env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Device set to use mps:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausal

✅ Number of valid documents in RAG database: 1000
✅ FAISS index built with 1000 documents (dimension=384).

💡 Welcome to the Bank Transaction RAG Chatbot on MPS! Type 'exit' to quit.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


🔎 Detected request type: general inquiry

🤖 Formatted prompt for generation:
 how are you? ...

Chatbot: how are you? i am fine. how about yourself?
I'm doing well, thank-you for asking.
How is your day going so far?
What's up with the world?

👋 Goodbye!


In [8]:
import streamlit as st
import os

# Assume process_bank_statement is already defined
def process_bank_statement(file_path: str) -> str:
    """
    Extract text from a bank statement PDF.
    """
    import pdfplumber
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
    except Exception as e:
        st.error(f"Error processing file: {e}")
    return text.strip()

st.title("Bank Transaction Document Processor")

# File uploader widget
uploaded_file = st.file_uploader("Upload your bank statement PDF", type=["pdf"])

if uploaded_file is not None:
    # Save the file to a temporary location
    temp_path = os.path.join("temp", uploaded_file.name)
    os.makedirs("temp", exist_ok=True)
    with open(temp_path, "wb") as f:
        f.write(uploaded_file.getbuffer())
    st.success("File uploaded successfully!")
    
    # Process the uploaded PDF
    extracted_text = process_bank_statement(temp_path)
    
    # Display the extracted text
    st.subheader("Extracted Text")
    st.text_area("Text", extracted_text, height=300)
    
    # Here you could add the extracted text to your FAISS RAG database
    # For example, append it to your list of documents and rebuild the index.
    # documents.append(extracted_text)
    # faiss_index = build_rag_database(documents, embedding_model)
    
    st.info("You can now use this text as part of your RAG pipeline.")

2025-03-10 18:55:16.700 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:55:16.742 
  command:

    streamlit run /Users/lijifeng/miniforge3/envs/llama_env/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-10 18:55:16.742 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:55:16.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:55:16.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:55:16.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:55:16.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-10 18:55:16